In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

dataset_path = "" 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
#     print(dirname)
    for filename in filenames:
#         print(os.path.join(dirname, filename))
        dataset_path = os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
model_directory = "/kaggle/input/segment-anything/pytorch/vit-h/1"
import sys
sys.path.append(model_directory)

In [3]:
# dataset_directory = "/kaggle/input/fitzpatrick/CategoizedFitz"
# import sys
# sys.path.append(dataset_directory)

In [4]:
# import glob
# for files in glob.glob(model_directory + "/*.*"):
#     print(files)

In [5]:
# print(sys.path)

In [8]:
!pip install --target=/kaggle/working/ segment_anything 

In [9]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

In [10]:
model_type = "vit_l"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint="/kaggle/input/segment-anything/pytorch/vit-l/1/model.pth")
sam.to(device=device)
sam.eval()
mask_generator = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=32,
    points_per_batch=256,
    pred_iou_thresh=0.90,
    stability_score_thresh=0.92,
    crop_n_layers=1,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=100,  # Requires open-cv to run post-processing,
    output_mode="binary_mask",
    
    
    
)

In [11]:
!pip install --target=/kaggle/working/ Flask==3.0.0 pyngrok==7.1.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 2.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.3/227.3 kB 15.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.1.5 which is incompatible.
dask-cuda 23.8.0 requires dask==2023.7.1, but you have dask 2024.4.0 which is incompatible.
dask-cuda 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you ha

In [12]:
!pip install --target=/kaggle/working/ flask-cors

  Using cached werkzeug-3.0.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached blinker-1.8.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached MarkupSafe-2.1.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.4 MB/s eta 0:00:00 0:00:01
Using cached blinker-1.8.2-py3-none-any.whl (9.5 kB)
Using cached click-8.1.7-py3-none-any.whl (97 kB)
Using cached itsdangerous-2.2.0-py3-none-any.whl (16 kB)
Using cached jinja2-3.1.4-py3-none-any.whl (133 kB)
Using cached werkzeug-3.0.3-py3-none-any.whl (227 kB)
Using cached MarkupSafe-2.1.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behav

In [13]:
from flask import Flask, request, send_file, jsonify
from flask_cors import CORS, cross_origin
import numpy as np
from io import BytesIO
from PIL import Image
import os
import cv2

In [14]:
ngrok_key = "2ezHPWIApfgO5FcXUIwxd7SIQva_5WUUwCMmBBMTuiew5nSoz"
port = 5000

In [15]:
from pyngrok import ngrok

In [16]:
ngrok.set_auth_token(ngrok_key)
ngrok.connect(port,domain="tidy-octopus-diverse.ngrok-free.app").public_url

'https://tidy-octopus-diverse.ngrok-free.app'

In [17]:
app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'

In [18]:
@app.route("/")
def hello():
  return "Hello World!! from anywhere in the world!"


In [19]:

@app.route('/segment', methods=['POST'])
def segment():
    if 'file' not in request.files:
        return 'No file part', 400
    file = request.files['file']
    if file.filename == '':
        return 'No selected file', 400
    if file:
        # Read the file into a numpy array using OpenCV
        filestr = file.read()
        # Convert the string data to numpy array
        npimg = np.frombuffer(filestr, np.uint8)
        # Load the numpy array as an OpenCV image
        img = cv2.imdecode(npimg, cv2.IMREAD_COLOR)

        # Process the image (here, we just save it as is)
        save_path = '/kaggle/working/temp_image.png'  # Specify the path and filename
        cv2.imwrite(save_path, img)
        
        
        image = cv2.imread(save_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        print(image)
        masks = mask_generator.generate(image)
        print(masks)
        data = masks

        # Thresholds for filtering masks
        lower_threshold = 0.0001  # Discard masks with less than 5% True values
        upper_threshold = 0.95  # Discard masks with more than 95% True values
        
            # Check if thresholds are provided in the form data and update accordingly
        if 'lower_threshold' in request.form:
            try:
                lower_threshold = float(request.form['lower_threshold'])
            except ValueError:
                return 'Invalid lower_threshold value', 400
        if 'upper_threshold' in request.form:
            try:
                upper_threshold = float(request.form['upper_threshold'])
            except ValueError:
                return 'Invalid upper_threshold value', 400
        # Filter out almost pure white or pure black masks
        filtered_masks = [segment['segmentation'] for segment in data if lower_threshold < np.mean(segment['segmentation']) < upper_threshold]
        print(filtered_masks)
        # Combine all remaining masks using OR operator
        if filtered_masks:
            combined_mask_or = np.logical_or.reduce(filtered_masks)
            # Convert the combined mask to 8-bit unsigned integer format for saving
            combined_mask_uint8 = (combined_mask_or * 255).astype(np.uint8)
            cv2.imwrite('/kaggle/working/combined_mask_or.png', combined_mask_uint8)
        else:
            return 'Filteration Threshes were not set appropriately', 400
        # Here, you can process the image if necessary

        # For simplicity, we return the same image
        return send_file("/kaggle/working/combined_mask_or.png", mimetype='image/png')

In [20]:
if __name__ == '__main__':
  app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off
[[[ 78  52  39]
  [ 94  68  55]
  [ 88  62  47]
  ...
  [247 230 222]
  [247 230 222]
  [247 230 222]]

 [[ 94  68  55]
  [ 96  70  57]
  [ 77  51  36]
  ...
  [247 230 222]
  [246 229 221]
  [246 229 221]]

 [[ 97  71  58]
  [ 83  57  44]
  [ 77  51  36]
  ...
  [246 229 221]
  [246 229 221]
  [246 229 221]]

 ...

 [[206 160 134]
  [207 161 135]
  [208 162 136]
  ...
  [ 61  24   5]
  [ 67  30  11]
  [ 76  40  18]]

 [[208 162 136]
  [209 163 137]
  [209 163 137]
  ...
  [ 62  25   6]
  [ 69  32  13]
  [ 79  43  21]]

 [[208 162 136]
  [209 163 137]
  [210 164 138]
  ...
  [ 63  26   7]
  [ 71  34  15]
  [ 80  44  22]]]
[{'segmentation': array([[False, False, False, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [F

In [ ]:
################################################